In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pickle

import os
import re
from collections import OrderedDict

import seaborn as sns
sns.set_theme(font_scale=1.5);

In [2]:
Earray   = np.round(np.logspace(np.log10(0.250), np.log10(10000), 100), 2)
h        = np.linspace(0, 499, 500);
binWidth = np.hstack([np.diff(Earray), np.diff(Earray)[-1]]);

def ProcessIoniData(data):
    
    lineNo = 1000
    
    nRuns  = int(len(data)/lineNo)
    
    mean = np.zeros([1000])
    std  = np.zeros([1000])
    for i in range(0, nRuns):
        
        mean += data.E.values[i*lineNo:(i+1)*lineNo]
        
        std  += (data.E.values[i*lineNo:(i+1)*lineNo])**2
        
    std -= (mean/nRuns)**2
    
    std[std < 0] = 0
    
    std = np.sqrt(1/(nRuns-1) * std)
    
    return mean, std

def ProcessSpectralData(data):
    
    lineNo = 1000
    
    nRuns  = int(len(data)/lineNo)
    
    data.fillna(data.mean(), inplace=True)

    mean = np.zeros([lineNo, 100]);
    std  = np.zeros([lineNo, 100]);
    for i in range(0, nRuns):
        mean += data.values[lineNo*i:lineNo*(i+1), :]
        std  += (data.values[lineNo*i:lineNo*(i+1), :])**2
        
    std -= (mean/nRuns)**2
    
    std[std < 0] = 0
    
    std = np.sqrt(1/(nRuns-1) * std)
    
    return mean/binWidth, std/binWidth

From sin(a$\alpha$) from $\alpha \in [0, 45^\circ]$,

$$N_{n=1} = 2 \pi \int_{0}^{45^\circ} sin^2(\alpha)~ d\alpha = 0.89660$$

$$\frac{1}{2\pi N_n} \frac{d^3 N}{dt ~dA ~d\Omega} = f_0 \rightarrow \frac{N}{2\pi N_n} = f_0$$

In [3]:
normFactor = 1e5 / 0.89660

In [6]:
def build_table(path, energyDist):

    files = os.listdir(path)
    
    D = OrderedDict()
    
    files = [f for f in files if energyDist in f]

    for file in files:
        tmp = file.split('_')
        particle = tmp[0]
        fileType = tmp[1]
        E = int(re.findall(r'\d+', file)[0])

        if fileType == "ene":
            fileType = "spectra"
            readIn = pd.read_csv(path + file, names=Earray)
            mean, std = np.divide(ProcessSpectralData(readIn), normFactor)
            
            if "mono" in path:
                mean[:, Earray > E+1] = 0
                std[:, Earray > E+1]  = 0
            
            mean = mean[:500, :]
            std  = std[:500, :]

        elif fileType == "dep":
            fileType = "ioni"
            readIn = pd.read_csv(path + file, names=['E'])
            mean, std = np.divide(ProcessIoniData(readIn), normFactor)
            mean = mean[:500]
            std  = std[:500]

        D[(particle, fileType, E)] = (mean, std)
        
    return D

    
D_mono = build_table("../data/lowCeiling/", "mono")
D_exp  = build_table("../data/lowCeiling/", "exp")

In [9]:
filename = "G4data_sin45_mono.pkl"

with open(filename, 'wb') as f:
    pickle.dump(D_mono, f, protocol=pickle.HIGHEST_PROTOCOL)

D_mono = pickle.load(open(filename, 'rb'))
    
filename = "G4data_sin45_exp.pkl"

with open(filename, 'wb') as f:
    pickle.dump(D_exp, f, protocol=pickle.HIGHEST_PROTOCOL)
    
D_exp = pickle.load(open(filename, 'rb'))

    
!cp G4data_sin45_mono.pkl G4data_sin45_exp.pkl ../../G4EPP-git/G4EPP/data/
    
'''
# Test
del D
    
with open(filename, 'rb') as f:
    D = pickle.load(f)
''';

In [ ]:
runList = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]

Sig = 10

plt.figure(figsize=(16,8)); plt.subplot(1,2,1); plt.grid(True, which='both')
for item in runList:
    plt.semilogx((D_mono[('electron', 'ioni', item)][0] + D_mono[('photon', 'ioni', item)][0]), 
                 h, label="%.0f keV" % item, linewidth=1);
    
    #plt.semilogx((D_mono[('electron', 'ioni', item)][0] + D_mono[('photon', 'ioni', item)][0]) + 
    #             Sig*(D_mono[('electron', 'ioni', item)][1] + D_mono[('photon', 'ioni', item)][1])
    #             , h, '--');
    #
    #plt.semilogx((D_mono[('electron', 'ioni', item)][0] + D_mono[('photon', 'ioni', item)][0]) - 
    #             Sig*(D_mono[('electron', 'ioni', item)][1] + D_mono[('photon', 'ioni', item)][1]), 
    #             h, '--', linewidth=1);

plt.xlabel('Deposited Energy [keV / km / cm$^{-2}$ sec$^{-1}$ sr$^{-1}$]');
plt.ylabel('Altitude [km]');
plt.legend();
plt.ylim(0, 300);
plt.xlim(1e-5, 1e3);
plt.title('Monoenergetic');


plt.subplot(1,2,2); plt.grid(True, which='both')
for item in runList:
    plt.semilogx((D_exp[('electron', 'ioni', item)][0] + D_exp[('photon', 'ioni', item)][0]), 
                 h, label="%.0f keV" % item, linewidth=1);
    
    #plt.semilogx((D_exp[('electron', 'ioni', item)][0] + D_exp[('photon', 'ioni', item)][0]) + 
    #             Sig*(D_exp[('electron', 'ioni', item)][1] + D_exp[('photon', 'ioni', item)][1])
    #             , h, '--');
    #
    #plt.semilogx((D_exp[('electron', 'ioni', item)][0] + D_exp[('photon', 'ioni', item)][0]) - 
    #             Sig*(D_exp[('electron', 'ioni', item)][1] + D_exp[('photon', 'ioni', item)][1]), 
    #             h, '--', linewidth=1);

plt.xlabel('Deposited Energy [keV / km / cm$^{-2}$ sec$^{-1}$ sr$^{-1}$]');
plt.ylabel('Altitude [km]');
plt.legend();
plt.ylim(0, 300);
plt.xlim(1e-5, 1e3);
plt.title('Exponential');

In [ ]:
runList = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]

X, Y = np.meshgrid(Earray, h)

from matplotlib.colors import LogNorm

Sig = 10

Erun = 500

plt.figure(figsize=(16,8)); plt.subplot(1,2,1); plt.grid(True, which='both')
for item in [Erun]:
    plt.pcolormesh(X, Y, (D_mono[('electron', 'spectra', item)][0] + D_mono[('photon', 'spectra', item)][0]), 
                 norm=LogNorm());
    
plt.xlabel('Deposited Energy [keV / km / cm$^{-2}$ sec$^{-1}$ sr$^{-1}$]');
plt.ylabel('Altitude [km]');
plt.ylim(0, 300);
plt.xscale('log');
plt.title('Monoenergetic');


plt.subplot(1,2,2); plt.grid(True, which='both')
for item in [Erun]:
    plt.pcolormesh(X, Y, (D_exp[('electron', 'spectra', item)][0] + D_exp[('photon', 'spectra', item)][0]), 
                 norm=LogNorm());
    
plt.xlabel('Deposited Energy [keV / km / cm$^{-2}$ sec$^{-1}$ sr$^{-1}$]');
plt.ylabel('Altitude [km]');
plt.ylim(0, 300);
plt.xscale('log');
plt.title('Exponential');